In [1]:
from itertools import combinations
from tqdm.auto import tqdm
import numpy as np
import sample as sampler
import utils
import attacker

## Generate puzzle with given error_rate

generate n isometrices(sketches) with noise. Assume the original template is w, then the output for each sketching is Matrix = SS(w + noise) and the correponding codeword as b satisfying Matrix * (w + noise) = b.

In [ ]:
# codeword parameters
dimension = 512
alpha = 2
n = 511
error_rate = 0.35    # angle(w + noise_1, w + noise_2) 
                    # approximate 2*arcsin(error_rate * \sqrt(2) / \sqrt(1 + error_rate^2) / 2)
print("Estimated angle between two noisy templates: {}".format(2 * np.arcsin(error_rate * np.sqrt(2) / np.sqrt(1 + error_rate**2) / 2) * 180 / np.pi))

In [ ]:
w, cs, isometric_matrixes, _ = sampler.generate_puzzle_n(dimension, alpha, error_rate=error_rate, n = n)

In [ ]:
# get the mean angle between each pair of templates, i.e. angle(w + noise_1, w + noise_2)

angles = []
ws = [M.T @ b for M, b in zip(isometric_matrixes, cs)]
for i, j in tqdm(combinations(range(len(ws)), 2)):
    angles.append(utils.get_angle_of_two_vectors(ws[i], ws[j]))

print("Test Mean Degree: {:.2f}{}".format(np.mean(angles), u'\u00b0'))

## solving original template if sampled matrix is correct

ensure the matrix generated by linear equation sampler is "correct" as in Definition 4.1 in paper.

svd solver

In [ ]:
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, cs, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="SVD", disable_tqdm=False, k_each_matrix=1)

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

lsa solver

In [ ]:
assume_vector, b = attacker.solve_puzzle_with_n_matrix_known_places(isometric_matrixes, cs, dimension, alpha, threshold=49, max_rtimes=10000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, error_rate = error_rate * 3.0)

In [ ]:
# check the result(b is whether equal to the first code word)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

## solving original template without preposition that sampled matrix is "correct"

In [ ]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000,
                                                        algorithm="SVD", disable_tqdm=False, k_each_matrix=1, scale=100.0)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))

In [ ]:
# toy example for small k = 2
assume_vector, b = attacker.solve_puzzle_with_n_matrix(isometric_matrixes, dimension, alpha, threshold=49, max_rtimes=1000, algorithm="LSA", disable_tqdm=False, k_each_matrix=1, scale=1000.0, error_rate=error_rate * 3.0)

In [ ]:
# check the result(b is whether equal to the first codeword)
print("The result is: ", np.allclose(b, cs[0]) or np.allclose(b, -cs[0]))